In [1]:
import transformers
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model
from datasets import load_dataset
from transformers import AutoTokenizer, Trainer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import os

/home/heichtai/Projects/tutorial-finetune-LLM/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Naming the run
model_name = 'microsoft/phi-2'
project = "Evol-Instruct-Code-80k-v1-10-precent"
base_model_name = model_name.split("/")[1]
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

In [3]:
# Load the dataset
huggingface_dataset_name = "nickrosh/Evol-Instruct-Code-80k-v1"
# For the sake of this example, we will only use the first 10% of the training set
train_dataset = load_dataset(huggingface_dataset_name, split="train[:10%]")
print(train_dataset)

Dataset({
    features: ['instruction', 'output'],
    num_rows: 7826
})


In [4]:
# Load the model in 4bit quantization mode to save memory
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    # This is the PR that adds gradient checkpointing. It is not merged yet. This kind of thing is the cost of being on the bleeding edge.
    revision="refs/pr/23",
)

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.92it/s]
You are using an old version of the checkpointing format that is deprecated (We will also silently ignore `gradient_checkpointing_kwargs` in case you passed it).Please update to the new format on your modeling file. To use the new format, you need to completely remove the definition of the method `_set_gradient_checkpointing` in your model.


In [5]:
# Load and config the tokenizer

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    padding_side="left",
    add_eos_token=True,
    use_fast=True,
)

# add special tokens for ChatML formatting and a pad token
tokenizer.add_tokens(["<|im_start|>", "<PAD>"])
tokenizer.pad_token = "<PAD>"
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))
# For the sake of memory and time, we will only use 64 tokens
tokenizer.model_max_length = 8
# resize model embeddings
model.resize_token_embeddings(
    new_num_tokens=len(tokenizer),
    pad_to_multiple_of=64)   # phi2 default is 64, see configuration_phi.py
model.config.eos_token_id = tokenizer.eos_token_id

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# Define the tokenizing function to tokenize the dataset
def tokenize_function(data_point):
    def get_prompt(inst):
        return f"""
        # System:
        You are a helpful AI assistant. Follow the instruction. 
        # INSTRUCTION:
        {inst}
        # CODE:
        """
    prompts = [get_prompt(inst) for inst in data_point['instruction']]
    data_point['input_ids'] = tokenizer(
        prompts,
        truncation=True,
        max_length=tokenizer.model_max_length,
        padding='max_length',
    ).input_ids
    data_point['labels'] = tokenizer(
        data_point['output'],
        truncation=True,
        max_length=tokenizer.model_max_length,
        padding='max_length',
    ).input_ids
    return data_point


tokenized_train_dataset = train_dataset.map(
    tokenize_function, batched=True, num_proc=os.cpu_count())

In [7]:
# Apply the Peft adapter
config = LoraConfig(
    r=1,
    lora_alpha=16,
    target_modules=[
        'Wqkv', 'out_proj'
    ],
    bias="none",
    lora_dropout=0.05,
    # because we added new tokens
    modules_to_save=["lm_head", "embed_tokens"],
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)

In [8]:
# We can see the number of trainable parameters are only few percent of the original model
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )


print_trainable_parameters(model)

trainable params: 129325184 || all params: 1646129408 || trainable%: 7.856319398189137


In [9]:
# Define the trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=0,
        per_device_train_batch_size=1,
        gradient_accumulation_steps=1,
        max_steps=100,
        learning_rate=2.5e-5,
        lr_scheduler_type="constant",
        logging_steps=50,
        dataloader_num_workers=4,
        bf16=True,
        optim="paged_adamw_8bit",
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=50,                # Save checkpoints every 50 steps
        report_to="tensorboard",
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, mlm=False),
)

In [10]:
# Train!
trainer.train()
model.save_pretrained(output_dir)

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,3.147800
100,0.309000


Checkpoint destination directory ./phi-2-Evol-Instruct-Code-80k-v1-10-precent/checkpoint-50 already exists and is non-empty.Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./phi-2-Evol-Instruct-Code-80k-v1-10-precent/checkpoint-100 already exists and is non-empty.Saving will proceed but saved results may be invalid.
